<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span><ul class="toc-item"><li><span><a href="#read-func" data-toc-modified-id="read-func-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>read func</a></span></li></ul></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Process-Data" data-toc-modified-id="Process-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Process Data</a></span><ul class="toc-item"><li><span><a href="#process-func" data-toc-modified-id="process-func-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>process func</a></span></li></ul></li><li><span><a href="#Submission" data-toc-modified-id="Submission-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Submission</a></span><ul class="toc-item"><li><span><a href="#sub-func" data-toc-modified-id="sub-func-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>sub func</a></span></li></ul></li><li><span><a href="#Explore-Data" data-toc-modified-id="Explore-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Explore Data</a></span></li></ul></div>

# Import

In [188]:
#coding=utf-8
import pandas as pd
import json
import codecs
import datetime
import CJNLP as cjn

## read func

In [141]:
def get_result_dataframe(filename):
    result_text = pd.read_json(filename, lines=True, encoding='utf-8')
    result_pd = pd.DataFrame()
    for i in range(len(result_text)):
        tmp_pd = pd.DataFrame.from_records(result_text['entities'][i])
        tmp_pd['newsId'] = result_text['newsId'][i]
        result_pd = result_pd.append(tmp_pd)    
    return result_pd

#读取结果文件内容为Dict格式
def get_json_list_from(filename='DATA_T_RESULT.txt'):
    result = []
    f = codecs.open(filename,'r', 'utf-8')
    for line in f.readlines():
        #print line
        rj = json.loads(line)
        result.append(rj)
    f.close()    
    return result

# Load Data

In [6]:
# load to pandas 
train_pd = pd.read_json("DATA_T.txt", lines=True, encoding='utf-8')
test_pd = pd.read_json("DATA_TEST.txt", lines=True, encoding='utf-8')
origin_result_pd = pd.read_json("DATA_T_RESULT.txt", lines=True, encoding='utf-8')
result_pd = get_result_dataframe("DATA_T_RESULT.txt")
train_pd['type'] = 'Train'
test_pd['type'] = 'Test'
fulldata = pd.concat([train_pd, test_pd], axis=0)

In [149]:
# load to json
train_json = get_json_list_from(filename='DATA_T.txt')
test_json =  get_json_list_from("DATA_TEST.txt")
result_json = get_json_list_from()

In [213]:
ch = []
if len(ch) == 0:
    print 1

1


# Process Data

## process func

In [202]:
#生成一条新的实体
def create_ent(eventLevel=u"负向", name="", digest="", keywords=""):
    ent = { "name":name, "digest": digest, "keywords":keywords, "eventLevel":eventLevel}
    return ent
#根据实体集合生成JSON
def create_json_byEntitiesSet(entities):
    ents = []
    for e in entities:
        ents.append(create_ent(name=e.decode('utf-8')))
    return ents
#生成一条新的新闻记录
def create_newsRecord(newsId, ent_json):
    newsRecord = {"newsId":newsId, "entities":ent_json}
    return newsRecord 

#得到识别命名实体后的数据 newsId ,entities
def get_process_data(data):
    result = []
    i = 1
    for news in data:
        print 'process news ', i
        i += 1
        ent_set = cjn.get_entity_P(unicode.encode(news['body']))
        ent_json = create_json_byEntitiesSet(ent_set)
        record = create_newsRecord(news['newsId'], ent_json)
        result.append(record)
    return result

In [196]:
pre_data = test_json[:2]
process_data = get_process_data(pre_data)

In [201]:
save_submission(process_data)

In [189]:
#提取命名实体
se = cjn.get_entity_P(unicode.encode(test_json[0]['body']))

# Submission

## sub func

In [200]:

def save_submission(results, filename='sub[{0}].txt'.format(datetime.datetime.now().strftime('%m-%d#%H-%M'))):
    f = codecs.open(filename, "w", "utf-8")
    for i in range(len(results)):
        tr = json.dumps(results[i],ensure_ascii=False, separators=(',',':'))
        f.write(tr+"\r\n")
    f.close()

In [142]:
test = get_json_list_from('submission1.txt')

In [133]:
sub = get_json_list_from()
for i in range(len(sub)):
    tmp = sub[i]['entities']
    for j in tmp:
        j['eventLevel'] = u'负向'
        j['digest']=u''
for i in range(len(sub)):
    sub[i]['newsId'] = test_json[i]['newsId']

In [136]:
for i in range(10):
    print i+1,'#',test_json[i]['newsId'], '<->',i+1,'#',sub[i]['newsId']

1 # 36a16da602af53883bb21f3059abd5809292d7bb <-> 1 # 36a16da602af53883bb21f3059abd5809292d7bb
2 # 3d14c54d1f8e766909f18792b6c37098b477e83d <-> 2 # 3d14c54d1f8e766909f18792b6c37098b477e83d
3 # 5fe3a97a944211cb2aca3364c11de3adae09b2e2 <-> 3 # 5fe3a97a944211cb2aca3364c11de3adae09b2e2
4 # d686885eb3959b67a2a359e1570578df0c30c86e <-> 4 # d686885eb3959b67a2a359e1570578df0c30c86e
5 # 705d6be02163afcf9972fb0bde0b52a7f462bb68 <-> 5 # 705d6be02163afcf9972fb0bde0b52a7f462bb68
6 # 31b4988c4f3fe4ef13519b32be46d15a71846225 <-> 6 # 31b4988c4f3fe4ef13519b32be46d15a71846225
7 # 7d1c144d501a455a24c89cf6f3e3209e69a09922 <-> 7 # 7d1c144d501a455a24c89cf6f3e3209e69a09922
8 # 46dd717455d442754642d377d76a6e06ef33dbe8 <-> 8 # 46dd717455d442754642d377d76a6e06ef33dbe8
9 # 793c728c634c0df681889a3edefb52b046faa7f4 <-> 9 # 793c728c634c0df681889a3edefb52b046faa7f4
10 # 2a01c1f37840edba50d59501a1c2c08b209f3345 <-> 10 # 2a01c1f37840edba50d59501a1c2c08b209f3345


In [137]:
save_submission(sub)

In [43]:
with open("r.json",'w') as json_file:
         json.dump(tjs,json_file,ensure_ascii=False)

In [34]:
n = create_newsRecord(origin_result_pd['newsId'][0], origin_result_pd['entities'][0])
n

{'entities': [{u'digest': u'\uff0c\u5176\u4e2d\uff0c\u540c\u4ec1\u70ed\u6c5e\u8d2d\u7269\u4e2d\u5fc3\u4ea7\u54c1\u62bd\u68c0\u5408\u683c',
   u'eventLevel': u'\u6b63\u5411',
   u'keywords': u'',
   u'name': u'\u540c\u4ec1\u70ed\u6c5e\u8d2d\u7269\u4e2d\u5fc3'},
  {u'digest': u'\uff0c\u5176\u4e2d\uff0c\u6cfd\u5e93\u4e07\u5bb6\u4e50\u8d85\u5e02\u4ea7\u54c1\u62bd\u68c0\u5408\u683c',
   u'eventLevel': u'\u6b63\u5411',
   u'keywords': u'',
   u'name': u'\u6cfd\u5e93\u4e07\u5bb6\u4e50\u8d85\u5e02'},
  {u'digest': u'??\u62bd\u68c0\u51fa\u76842\u6279\u6b21\u4e0d\u5408\u7684\u519c\u4ea7\u54c1\u662f\uff0c\u4f5b\u5c71\u5e02\u9ad8\u660e\u533a\u8377\u57ce\u82cf\u7389\u840d\u852c\u83dc\u6863\u9500\u552e\u7684\u82b9\u83dc\uff0c\u4e0d\u5408\u683c\u9879\u76ee\u4e3a\u6bd2\u6b7b\u8731\uff0c\u53e6\u4e00\u6279\u6b21\u662f\u9ec4\u7389\u534e\u9500\u552e\u7684\u9cae\u9c7c\uff0c\u4e0d\u5408\u683c\u9879\u76ee\u4e3a\u5b54\u96c0\u77f3\u7eff\u3002',
   u'eventLevel': u'\u8d1f\u5411',
   u'keywords': u'',
   u'name'

In [24]:
origin_result_pd[:2]

,entities,newsId
0,"[{u'eventLevel': u'正向', u'keywords': u'', u'na...",82ca95bff6cbef092d4da7e8fceb1c38e923c54a
1,"[{u'eventLevel': u'负向', u'keywords': u'', u'na...",60268320291c75350c6b1f05a807f67111109d92


In [ ]:
{
    "newsId": "训练集中舆情标识ID",
    "entities": [
    {
        "name": "机构实体1",
        "digest": "文摘",
        "keywords": "关键词，多个词用 | 隔开",
        "eventLevel": "正向"
    },
    {
        "name": "机构实体2",
        "digest": "文摘",
        "keywords": "关键词，多个词用 | 隔开",
        "eventLevel": "负向"
    }]
}


In [20]:
sub = {}
sub['newsId'] = '1111'
en1 =     {
        "name": "机构实体1",
        "digest": "文摘",
        "keywords": "关键词，多个词用 | 隔开",
        "eventLevel": "正向"
    }
ens = [en1, en1]
sub['entities'] = ens
sub

{'entities': [{'digest': '\xe6\x96\x87\xe6\x91\x98',
   'eventLevel': '\xe6\xad\xa3\xe5\x90\x91',
   'keywords': '\xe5\x85\xb3\xe9\x94\xae\xe8\xaf\x8d\xef\xbc\x8c\xe5\xa4\x9a\xe4\xb8\xaa\xe8\xaf\x8d\xe7\x94\xa8 | \xe9\x9a\x94\xe5\xbc\x80',
   'name': '\xe6\x9c\xba\xe6\x9e\x84\xe5\xae\x9e\xe4\xbd\x931'},
  {'digest': '\xe6\x96\x87\xe6\x91\x98',
   'eventLevel': '\xe6\xad\xa3\xe5\x90\x91',
   'keywords': '\xe5\x85\xb3\xe9\x94\xae\xe8\xaf\x8d\xef\xbc\x8c\xe5\xa4\x9a\xe4\xb8\xaa\xe8\xaf\x8d\xe7\x94\xa8 | \xe9\x9a\x94\xe5\xbc\x80',
   'name': '\xe6\x9c\xba\xe6\x9e\x84\xe5\xae\x9e\xe4\xbd\x931'}],
 'newsId': '1111'}

In [14]:
test = [{"username":"测试","age":16},(2,3),1]
print type(test)
python_to_json = json.dumps(test,ensure_ascii=False)
print python_to_json
print type(python_to_json)

# 将json对象转换成python对象
json_to_python = json.loads(python_to_json)
print json_to_python
print type(json_to_python)

<type 'list'>
[{"username": "测试", "age": 16}, [2, 3], 1]
<type 'str'>
[{'username': u'\u6d4b\u8bd5', 'age': 16}, [2, 3], 1]
<type 'list'>


# Explore Data

In [18]:
result_pd['eventLevel'].value_counts()

负向    6831
正向    5051
中性    2857
Name: eventLevel, dtype: int64

In [20]:
result_pd[result_pd['newsId']=='60268320291c75350c6b1f05a807f67111109d92']

,digest,eventLevel,keywords,name,newsId
0,，其中，厦门飘安医疗器械有限公司产品抽检不合格,负向,,厦门飘安医疗器械有限公司,60268320291c75350c6b1f05a807f67111109d92
1,，其中，尤溪县医院产品抽检不合格,负向,,尤溪县医院,60268320291c75350c6b1f05a807f67111109d92
2,中国质量新闻网讯 2017年9月26日，福建省食品药品监管局官网发布《福建省医疗器械质量公告...,中性,,福建省食药监局,60268320291c75350c6b1f05a807f67111109d92


In [55]:
t = result_pd[['name', 'eventLevel']].drop_duplicates()
t.to_csv('result.csv', index=False)
t.to_excel('result.xls', index=False)

In [44]:
namet = result_pd['name'].value_counts()
accept_count = pd.DataFrame({'org':namet.index, 'COUNT':namet.values})
accept_count['COUNT'].value_counts()

1     9410
2      866
3      392
4      151
5       76
6       31
7       29
8       17
9       13
13       6
10       6
14       5
11       5
12       4
16       4
25       3
18       3
17       2
27       2
15       2
30       2
19       1
20       1
29       1
22       1
23       1
Name: COUNT, dtype: int64

In [21]:
t = result_pd['newsId'].value_counts()
accept_count = pd.DataFrame({'newsId':t.index, 'COUNT':t.values})
accept_count

,COUNT,newsId
0,150,e33f54cf90e1a8e8fc5f361a29d1006f13fbdccd
1,127,ff4c00f192d98958c413b2b8e6b35ffdfdd850a5
2,108,d13389159f5e0afe6bc7411926f43dd768ede596
3,105,5fa69f25de385ab99ec4230c57c0f224dea5208a
4,95,f07a5e6b4f15e4ccf9d8e37441a3c9be4db7aa09
5,90,001fc849b8ae0b906be024bc6b9808c656fe4017
6,88,51702d2015945fb1da6e16b1433b9b7e87b0841c
7,87,9975abf902034a3be6b84809a76298d657060af6
8,85,8a5b7c1701f131fb66f57fde47300aba3c92e152
9,83,dbb1152d17e56db725718a0d6ae67710cbbf6ad6


In [170]:
acc_pd

,digest,eventLevel,keywords,name,newsId
0,，其中，同仁热汞购物中心产品抽检合格,正向,,同仁热汞购物中心,82ca95bff6cbef092d4da7e8fceb1c38e923c54a
1,，其中，泽库万家乐超市产品抽检合格,正向,,泽库万家乐超市,82ca95bff6cbef092d4da7e8fceb1c38e923c54a
2,??抽检出的2批次不合的农产品是，佛山市高明区荷城苏玉萍蔬菜档销售的芹菜，不合格项目为毒死蜱...,负向,,荷城苏玉萍蔬菜档,82ca95bff6cbef092d4da7e8fceb1c38e923c54a
3,，其中，门源县华东购物中心有限公司产品抽检合格,正向,,门源县华东购物中心有限公司,82ca95bff6cbef092d4da7e8fceb1c38e923c54a
4,中国质量新闻网讯 8月25日，青海省食品药品监督管理局官网发布《食品安全监督抽检信息通告 2...,中性,,青海省食药监局,82ca95bff6cbef092d4da7e8fceb1c38e923c54a
5,，其中，黄南世纪华联商贸有限公司产品抽检合格,正向,,黄南世纪华联商贸有限公司,82ca95bff6cbef092d4da7e8fceb1c38e923c54a


In [ ]:
def translate()

In [149]:
pd1 = pd.DataFrame.from_records(result_text['entities'][0])

In [150]:
pd2 = pd.DataFrame.from_records(result_text['entities'][1])

In [154]:
pd3 = pd.concat([pd1, pd2], axis=0,)

In [156]:
pd1.append(pd2)

,digest,eventLevel,keywords,name
0,，其中，同仁热汞购物中心产品抽检合格,正向,,同仁热汞购物中心
1,，其中，泽库万家乐超市产品抽检合格,正向,,泽库万家乐超市
2,??抽检出的2批次不合的农产品是，佛山市高明区荷城苏玉萍蔬菜档销售的芹菜，不合格项目为毒死蜱...,负向,,荷城苏玉萍蔬菜档
3,，其中，门源县华东购物中心有限公司产品抽检合格,正向,,门源县华东购物中心有限公司
4,中国质量新闻网讯 8月25日，青海省食品药品监督管理局官网发布《食品安全监督抽检信息通告 2...,中性,,青海省食药监局
5,，其中，黄南世纪华联商贸有限公司产品抽检合格,正向,,黄南世纪华联商贸有限公司
0,，其中，厦门飘安医疗器械有限公司产品抽检不合格,负向,,厦门飘安医疗器械有限公司
1,，其中，尤溪县医院产品抽检不合格,负向,,尤溪县医院
2,中国质量新闻网讯 2017年9月26日，福建省食品药品监管局官网发布《福建省医疗器械质量公告...,中性,,福建省食药监局


In [155]:
pd3

,digest,eventLevel,keywords,name
0,，其中，同仁热汞购物中心产品抽检合格,正向,,同仁热汞购物中心
1,，其中，泽库万家乐超市产品抽检合格,正向,,泽库万家乐超市
2,??抽检出的2批次不合的农产品是，佛山市高明区荷城苏玉萍蔬菜档销售的芹菜，不合格项目为毒死蜱...,负向,,荷城苏玉萍蔬菜档
3,，其中，门源县华东购物中心有限公司产品抽检合格,正向,,门源县华东购物中心有限公司
4,中国质量新闻网讯 8月25日，青海省食品药品监督管理局官网发布《食品安全监督抽检信息通告 2...,中性,,青海省食药监局
5,，其中，黄南世纪华联商贸有限公司产品抽检合格,正向,,黄南世纪华联商贸有限公司
0,，其中，厦门飘安医疗器械有限公司产品抽检不合格,负向,,厦门飘安医疗器械有限公司
1,，其中，尤溪县医院产品抽检不合格,负向,,尤溪县医院
2,中国质量新闻网讯 2017年9月26日，福建省食品药品监管局官网发布《福建省医疗器械质量公告...,中性,,福建省食药监局


In [75]:
result_string = open("DATA_T_RESULT.txt").read()
result_dumps = json.dumps(result_string)
result_json = json.loads(result_dumps)

In [106]:
string_json = json.loads(result_string)

ValueError: Extra data: line 2 column 1 - line 2001 column 1 (char 1386 - 3912677)

In [103]:
yj = yaml.safe_load(result_dumps)

In [105]:
yj[0]

u'{'

In [91]:
tmp = pd.read_json(result_text['entities'][0], lines=True, encoding='utf-8')

NameError: name 'string' is not defined

In [92]:
result_text['entities'][0]

[{u'digest': u'\uff0c\u5176\u4e2d\uff0c\u540c\u4ec1\u70ed\u6c5e\u8d2d\u7269\u4e2d\u5fc3\u4ea7\u54c1\u62bd\u68c0\u5408\u683c',
  u'eventLevel': u'\u6b63\u5411',
  u'keywords': u'',
  u'name': u'\u540c\u4ec1\u70ed\u6c5e\u8d2d\u7269\u4e2d\u5fc3'},
 {u'digest': u'\uff0c\u5176\u4e2d\uff0c\u6cfd\u5e93\u4e07\u5bb6\u4e50\u8d85\u5e02\u4ea7\u54c1\u62bd\u68c0\u5408\u683c',
  u'eventLevel': u'\u6b63\u5411',
  u'keywords': u'',
  u'name': u'\u6cfd\u5e93\u4e07\u5bb6\u4e50\u8d85\u5e02'},
 {u'digest': u'??\u62bd\u68c0\u51fa\u76842\u6279\u6b21\u4e0d\u5408\u7684\u519c\u4ea7\u54c1\u662f\uff0c\u4f5b\u5c71\u5e02\u9ad8\u660e\u533a\u8377\u57ce\u82cf\u7389\u840d\u852c\u83dc\u6863\u9500\u552e\u7684\u82b9\u83dc\uff0c\u4e0d\u5408\u683c\u9879\u76ee\u4e3a\u6bd2\u6b7b\u8731\uff0c\u53e6\u4e00\u6279\u6b21\u662f\u9ec4\u7389\u534e\u9500\u552e\u7684\u9cae\u9c7c\uff0c\u4e0d\u5408\u683c\u9879\u76ee\u4e3a\u5b54\u96c0\u77f3\u7eff\u3002',
  u'eventLevel': u'\u8d1f\u5411',
  u'keywords': u'',
  u'name': u'\u8377\u57ce\u82cf\u

                                                   0
0  {"newsId":"82ca95bff6cbef092d4da7e8fceb1c38e92...


In [35]:
import json
a = json.dumps(rjson)

In [44]:
for i in result_text['entities'][0]:
    print i

{u'eventLevel': u'\u6b63\u5411', u'keywords': u'', u'name': u'\u540c\u4ec1\u70ed\u6c5e\u8d2d\u7269\u4e2d\u5fc3', u'digest': u'\uff0c\u5176\u4e2d\uff0c\u540c\u4ec1\u70ed\u6c5e\u8d2d\u7269\u4e2d\u5fc3\u4ea7\u54c1\u62bd\u68c0\u5408\u683c'}
{u'eventLevel': u'\u6b63\u5411', u'keywords': u'', u'name': u'\u6cfd\u5e93\u4e07\u5bb6\u4e50\u8d85\u5e02', u'digest': u'\uff0c\u5176\u4e2d\uff0c\u6cfd\u5e93\u4e07\u5bb6\u4e50\u8d85\u5e02\u4ea7\u54c1\u62bd\u68c0\u5408\u683c'}
{u'eventLevel': u'\u8d1f\u5411', u'keywords': u'', u'name': u'\u8377\u57ce\u82cf\u7389\u840d\u852c\u83dc\u6863', u'digest': u'??\u62bd\u68c0\u51fa\u76842\u6279\u6b21\u4e0d\u5408\u7684\u519c\u4ea7\u54c1\u662f\uff0c\u4f5b\u5c71\u5e02\u9ad8\u660e\u533a\u8377\u57ce\u82cf\u7389\u840d\u852c\u83dc\u6863\u9500\u552e\u7684\u82b9\u83dc\uff0c\u4e0d\u5408\u683c\u9879\u76ee\u4e3a\u6bd2\u6b7b\u8731\uff0c\u53e6\u4e00\u6279\u6b21\u662f\u9ec4\u7389\u534e\u9500\u552e\u7684\u9cae\u9c7c\uff0c\u4e0d\u5408\u683c\u9879\u76ee\u4e3a\u5b54\u96c0\u77f3\u7eff\

In [58]:
result_text['entities'][0][0]

{u'digest': u'\uff0c\u5176\u4e2d\uff0c\u540c\u4ec1\u70ed\u6c5e\u8d2d\u7269\u4e2d\u5fc3\u4ea7\u54c1\u62bd\u68c0\u5408\u683c',
 u'eventLevel': u'\u6b63\u5411',
 u'keywords': u'',
 u'name': u'\u540c\u4ec1\u70ed\u6c5e\u8d2d\u7269\u4e2d\u5fc3'}

In [47]:
tmpjson = json.load(result_text['entities'][0][0])

AttributeError: 'dict' object has no attribute 'read'

In [93]:
data1 = {'b': 789, 'c': 456, 'a': 123}
encode_json = json.dumps(data1)
print type(encode_json), encode_json

decode_json = json.loads(encode_json)
print type(decode_json)
print decode_json['a']
print decode_json

<type 'str'> {"a": 123, "c": 456, "b": 789}
<type 'dict'>
123
{'a': 123, 'c': 456, 'b': 789}


In [98]:
encode_json

'{"a": 123, "c": 456, "b": 789}'

In [74]:
retv = json.dumps(d1)
js_retv = json.loads(result_json)

In [72]:
js_retv['k1']

'v1'